# Script - Requêtes SQL
## Ayoub Errhamani - Rachid Sahli
### BUT SD FA EMS 32 - SAE NoSQL

### Import des librairies

In [1]:
import sqlite3
import pandas

### Création de la connexion

In [38]:
conn = sqlite3.connect("ClassicModel.sqlite")

In [39]:
# Récupération du contenu de Customers avec une requête SQL
customers = pandas.read_sql_query("SELECT * FROM Customers;", conn)
print(customers)

     customerNumber                    customerName contactLastName  \
0               103               Atelier graphique         Schmitt   
1               112              Signal Gift Stores            King   
2               114      Australian Collectors, Co.        Ferguson   
3               119               La Rochelle Gifts         Labrune   
4               121              Baane Mini Imports      Bergulfsen   
..              ...                             ...             ...   
117             486    Motor Mint Distributors Inc.       Hernandez   
118             487        Signal Collectibles Ltd.          Taylor   
119             489  Double Decker Gift Stores, Ltd           Hardy   
120             495            Diecast Collectables          Franco   
121             496               Kelly's Gift Shop         Snowden   

    contactFirstName           phone                  addressLine1  \
0             Carine      40.32.2555                54, rue Royale   
1      

### Requête 1 - Lister les clients n’ayant jamais effecuté une commande

In [40]:
pandas.read_sql_query("""SELECT
                                C.customerNumber,
                                C.customerName
                                FROM Customers C LEFT JOIN Orders O
                                ON C.customerNumber = O.customerNumber
                                WHERE O.orderNumber IS NULL; """, conn)

,customerNumber,customerName
0,125,Havel & Zbyszek Co
1,168,American Souvenirs Inc
2,169,Porto Imports Co.
3,206,"Asian Shopping Network, Co"
4,223,Natürlich Autos
5,237,ANG Resellers
6,247,Messner Shopping Network
7,273,"Franken Gifts, Co"
8,293,BG&E Collectables
9,303,Schuyler Imports


### Requête 2 - Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci

In [37]:
pandas.read_sql_query("""SELECT
                                E.employeeNumber,
                                E.firstName,
                                E.lastName,
                                COUNT(DISTINCT C.customerNumber) AS TotalCustomer,
                                COUNT(DISTINCT O.orderNumber) AS TotalOrder,
                                COALESCE(SUM(P.amount), 0) AS TotalOrderAmount
                            FROM 
                            Employees E
                            LEFT JOIN 
                            Customers C ON E.employeeNumber = C.salesRepEmployeeNumber
                            LEFT JOIN 
                            Orders O ON C.customerNumber = O.customerNumber
                            LEFT JOIN 
                            Payments P ON C.customerNumber = P.customerNumber
                            GROUP BY 
                            E.employeeNumber, E.firstName, E.lastName
                            ORDER BY 
                            E.employeeNumber; """,conn)

,employeeNumber,firstName,lastName,TotalCustomer,TotalOrder,TotalOrderAmount
0,1002,Diane,Murphy,0,0,0.00
1,1056,Mary,Patterson,0,0,0.00
2,1076,Jeff,Firrelli,0,0,0.00
3,1088,William,Patterson,0,0,0.00
4,1102,Gerard,Bondur,0,0,0.00
5,1143,Anthony,Bow,0,0,0.00
6,1165,Leslie,Jennings,6,34,12674066.13
7,1166,Leslie,Thompson,6,14,943442.48
8,1188,Julie,Firrelli,6,14,1035043.99
9,1216,Steve,Patterson,6,18,1545990.08


### Requête 3 - Idem pour chaque bureau (nombre de clients, nombre de commandes et montant total), avec en plus le nombre de clients d’un pays différent, s’il y en a

In [9]:
pandas.read_sql_query("""SELECT
                                    DISTINCT(O.officeCode),
                                    COUNT(DISTINCT C.customerNumber) as TotalCustomer,
                                    COUNT(DISTINCT X.orderNumber) as TotalOrder,
                                    SUM(D.priceEach * D.quantityOrdered) as TotalamountOrder,
                                    O.city,
                                    O.country as OfficeCountry,
                                    COUNT(DISTINCT CASE
                                                WHEN C.country != O.country THEN C.CustomerNumber
                                                ELSE NULL
                                    END) AS CustomerfromdifferentCountry   
                                    FROM Offices O
                                    LEFT JOIN Employees E ON O.officeCode = E.officeCode
                                    LEFT JOIN Customers C ON E.employeeNumber = C.salesRepEmployeeNumber
                                    LEFT JOIN Orders X ON C.customerNumber = X.customerNumber
                                    LEFT JOIN OrderDetails D ON X.orderNumber = D.orderNumber
                                    GROUP BY O.officeCode;""",conn)

,officeCode,TotalCustomer,TotalOrder,TotalamountOrder,city,OfficeCountry,CustomerfromdifferentCountry
0,1.0,12,48,1588293.29,San Francisco,USA,0
1,2.0,12,32,987774.17,Boston,USA,0
2,3.0,15,39,1275993.95,NYC,USA,3
3,4.0,29,106,3404055.56,Paris,France,17
4,5.0,5,16,503957.58,Tokyo,Japan,3
5,6.0,10,38,1281705.83,Sydney,Australia,5
6,7.0,17,47,1604168.80,London,UK,12


### Requête 4 - Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents

In [10]:
pandas.read_sql_query("""SELECT
                                    P.productCode,
                                    P.productName,
                                    COUNT(DISTINCT D.orderNumber) as TotalOrder,
                                    SUM(D.quantityOrdered) as Totalquantity,
                                    COUNT(DISTINCT C.customerNumber) as TotalCustomer
                           
                            FROM Products P
                            LEFT JOIN OrderDetails D ON P.productCode = D.productCode
                            LEFT JOIN Orders O ON D.orderNumber = O.orderNumber
                            LEFT JOIN Customers C ON O.customerNumber = C.customerNumber
                            GROUP BY P.productCode, P.productName; """,conn)

,productCode,productName,TotalOrder,Totalquantity,TotalCustomer
0,S10_1678,1969 Harley Davidson Ultimate Chopper,28,1026.0,26
1,S10_1949,1952 Alpine Renault 1300,28,961.0,27
2,S10_2016,1996 Moto Guzzi 1100i,28,999.0,26
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,28,985.0,25
4,S10_4757,1972 Alfa Romeo GTA,28,1000.0,27
...,...,...,...,...,...
105,S700_3505,The Titanic,27,952.0,22
106,S700_3962,The Queen Mary,27,883.0,24
107,S700_4002,American Airlines: MD-11S,28,1073.0,26
108,S72_1253,Boeing X-32A JSF,28,960.0,27


### Requête 5 - Donner le nombre de commande pour chaque pays, ainsi que le montant total des commandes et le montant total payé : on veut conserver les clients n’ayant jamais commandé dans le résultat final

In [11]:
pandas.read_sql_query("""SELECT
                                    DISTINCT(C.country) as Country,
                                    COUNT(DISTINCT D.orderNumber) as TotalOrder,
                                    SUM(D.priceEach * D.quantityOrdered) as TotalOrderAmount,
                                    SUM(P.amount) as Totalpayment
                                
                            FROM OrderDetails D
                            LEFT JOIN Orders O ON D.orderNumber = O.orderNumber
                            FULL JOIN Customers C ON O.customerNumber = C.customerNumber
                            LEFT JOIN Payments P ON C.customerNumber = P.customerNumber        
                            GROUP BY C.country; """,conn)

,Country,TotalOrder,TotalOrderAmount,Totalpayment
0,Australia,19,2182269.38,2.482541e+07
1,Austria,7,606187.59,4.090982e+06
2,Belgium,7,283705.44,1.931535e+06
3,Canada,7,448157.12,4.487022e+06
4,Denmark,7,781357.50,7.001114e+06
5,Finland,9,988745.73,1.009620e+07
6,France,37,3160296.75,3.141444e+07
7,Germany,7,576293.44,4.971661e+06
8,Hong Kong,2,48784.36,7.805498e+05
9,Ireland,2,115512.86,9.241029e+05


### Requête 6 - On veut la table de contigence du nombre de commande entre la ligne de produits et le pays du client

In [21]:
pandas.read_sql_query("""SELECT 
                                c.country,
                                p.productLine,
                                COUNT(DISTINCT o.orderNumber) AS nb_commandes
                        FROM Customers c
                        JOIN Orders o ON c.customerNumber = o.customerNumber
                        JOIN OrderDetails d ON o.orderNumber = d.orderNumber
                        JOIN Products p ON d.productCode = p.productCode
                        GROUP BY c.country, p.productLine
                        ORDER BY c.country, p.productLine;""", conn)

,country,productLine,nb_commandes
0,Australia,Classic Cars,12
1,Australia,Motorcycles,6
2,Australia,Planes,5
3,Australia,Ships,2
4,Australia,Trains,1
...,...,...,...
121,USA,Planes,22
122,USA,Ships,25
123,USA,Trains,15
124,USA,Trucks and Buses,27


### Requête 7 - On veut la même table croisant la ligne de produits et le pays du client, mais avec le montant total payé dans chaque cellule

In [13]:
pandas.read_sql_query("""SELECT 
                                P.productLine,                               
                                SUM(PA.amount) as Totalamount,
                                C.country as Country

                            FROM Orders O
                            JOIN OrderDetails D ON O.orderNumber = D.orderNumber
                            JOIN Products P ON D.productCode = P.productCode
                            JOIN Customers C ON O.customerNumber = C.customerNumber
                            JOIN Payments PA ON C.customerNumber = PA.customerNumber
                            GROUP BY P.productLine, C.country;""",conn)

,productLine,Totalamount,Country
0,Classic Cars,7504795.97,Australia
1,Classic Cars,1884419.42,Austria
2,Classic Cars,166880.87,Belgium
3,Classic Cars,774924.01,Canada
4,Classic Cars,3678313.22,Denmark
...,...,...,...
121,Vintage Cars,1941227.28,Singapore
122,Vintage Cars,39878490.66,Spain
123,Vintage Cars,1377094.16,Sweden
124,Vintage Cars,4673162.78,UK


### Requête 8 - Donner les 10 produits pour lesquels la marge moyenne est la plus importante (cf buyPrice et priceEach)

In [14]:
pandas.read_sql_query("""SELECT
                                    P.productCode,
                                    P.productName,
                                    AVG(priceEach-buyprice) as AverageMargin
                                
                                FROM Products P
                                LEFT JOIN OrderDetails D ON P.productCode = D.productCode
                                GROUP BY P.productCode
                                ORDER BY AverageMargin DESC
                                LIMIT 10;""",conn)

,productCode,productName,AverageMargin
0,S10_1949,1952 Alpine Renault 1300,99.006429
1,S10_4698,2003 Harley-Davidson Eagle Drag Bike,95.235000
2,S18_3232,1992 Ferrari 360 Spider red,83.334906
3,S12_2823,2002 Suzuki XREO,83.201429
4,S18_2795,1928 Mercedes-Benz SSK,82.696786
5,S12_1108,2001 Ferrari Enzo,81.043704
6,S12_3891,1969 Ford Falcon,77.335926
7,S18_3685,1948 Porsche Type 356 Roadster,72.636800
8,S18_2870,1999 Indy 500 Monte Carlo SS,71.794400
9,S18_1749,1917 Grand Touring Sedan,70.432800


### Requête 9 - Lister les produits (avec le nom et le code du client) qui ont été vendus à perte :
            - Si un produit a été dans cette situation plusieurs fois, il doit apparaître plusieurs fois,
            - Une vente à perte arrive quand le prix de vente est inférieur au prix d’achat

In [22]:
pandas.read_sql_query("""SELECT
                                    P.productCode,
                                    P.productName,
                                    C.customerName,
                                    C.customerNumber,
                                    D.priceEach,
                                    P.buyPrice
                            FROM Products P
                            LEFT JOIN OrderDetails D ON P.productCode = D.productCode
                            LEFT JOIN Orders O ON D.ordernumber = O.ordernumber
                            LEFT JOIN Customers C ON O.customernumber = C.customernumber
                            WHERE D.priceEach < P.buyPrice;""",conn)

,productCode,productName,customerName,customerNumber,priceEach,buyPrice
0,S10_4962,1962 LanciaA Delta 16V,Online Diecast Creations Co.,363,61.99,103.42
1,S18_2957,1934 Ford V8 Coupe,Online Diecast Creations Co.,363,29.87,34.35
2,S18_3136,18th Century Vintage Horse Carriage,Online Diecast Creations Co.,363,47.04,60.74
3,S12_3148,1969 Corvair Monza,Vitachrome Inc.,181,54.33,89.14
4,S18_2319,1964 Mercedec Tour Bus,Vitachrome Inc.,181,37.48,74.86
...,...,...,...,...,...,...
74,S10_4962,1962 LanciaA Delta 16V,"Anna's Decorations, Ltd",276,46.90,103.42
75,S12_1666,1958 Setra Bus,"Anna's Decorations, Ltd",276,63.20,77.90
76,S18_2949,1913 Ford Model T Speedster,"Anna's Decorations, Ltd",276,45.25,60.78
77,S18_2238,1998 Chrysler Plymouth Prowler,"Down Under Souveniers, Inc",323,69.81,101.51


### Requête 10 - (bonus) Lister les clients pour lesquels le montant total payé est supérieur aux montants totals des achats

In [42]:
pandas.read_sql_query("""SELECT
                                     C.customerNumber,
                                     C.customerName,
                                     P.amount as Totalamount,
                                     D.priceEach*D.quantityOrdered as TotalBuying
                                FROM Customers C
                                LEFT JOIN Orders O ON C.customerNumber = O.customerNumber
                                LEFT JOIN OrderDetails D ON O.orderNumber = D.orderNumber
                                LEFT JOIN Payments P ON C.customerNumber = P.customerNumber
                                WHERE Totalamount > TotalBuying
                                GROUP BY C.customerNumber;""",conn)

,customerNumber,customerName,Totalamount,TotalBuying
0,103,Atelier graphique,5307.98,3073.72
1,112,Signal Gift Stores,14449.61,2856.00
2,114,"Australian Collectors, Co.",50397.66,2793.86
3,119,La Rochelle Gifts,20719.91,4177.35
4,121,Baane Mini Imports,54702.00,5404.62
...,...,...,...,...
93,486,Motor Mint Distributors Inc.,7287.24,4379.18
94,487,Signal Collectibles Ltd.,34100.03,5907.50
95,489,"Double Decker Gift Stores, Ltd",27541.82,3854.24
96,495,Diecast Collectables,63730.78,4076.19


### Fermeture de la connexion

In [43]:
conn.close()